In [14]:
 # Pandas
import pandas as pd
from pathlib import Path
Path('my_data.db').touch()


# SQL Alchemy
from sqlalchemy import create_engine

database_path = "Hotel Reservations.csv"

In [18]:
# Import sqlite and connect to engine
import sqlite3
conn = sqlite3.connect('my_data.db')
c = conn.cursor()

In [19]:
c.execute('''CREATE TABLE reservations (booking_id text, no_of_adults int, no_of_children int,
                                        no_of_weekend_nights int, no_of_week_nights int, type_of_meal_plan text, 
                                        required_car_parking_space int, room_type_reserved text, lead_time text,
                                        arrival_year int, arrival_month int, arrival_date int, market_segment_type text,
                                        repeated_guest int, no_of_previous_cancellations int, no_of_previous_bookings_not_canceled int,
                                        avg_price_per_room int, no_of_special_requests int, booking_status text)''')

In [20]:
# Read in csv file and convert to sqlite database
reservations = pd.read_csv('Hotel Reservations.csv')
reservations.to_sql('reservations', conn, if_exists='append', index=False)

36275

In [22]:

data = pd.read_sql("SELECT * FROM reservations", conn)

In [23]:
data.head()

,booking_id,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_year,arrival_month,arrival_date,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,INN00001,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,2017,10,2,Offline,0,0,0,65.00,0,Not_Canceled
1,INN00002,2,0,2,3,Not Selected,0,Room_Type 1,5,2018,11,6,Online,0,0,0,106.68,1,Not_Canceled
2,INN00003,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2018,2,28,Online,0,0,0,60.00,0,Canceled
3,INN00004,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,2018,5,20,Online,0,0,0,100.00,0,Canceled
4,INN00005,2,0,1,1,Not Selected,0,Room_Type 1,48,2018,4,11,Online,0,0,0,94.50,0,Canceled


In [28]:
data_clean = pd.read_sql("SELECT no_of_adults, no_of_children, no_of_weekend_nights, no_of_week_nights, type_of_meal_plan, required_car_parking_space, room_type_reserved, lead_time, arrival_month, market_segment_type, repeated_guest, no_of_previous_cancellations, no_of_previous_bookings_not_canceled, avg_price_per_room, no_of_special_requests, booking_status FROM reservations", conn)
data_clean.head()

,no_of_adults,no_of_children,no_of_weekend_nights,no_of_week_nights,type_of_meal_plan,required_car_parking_space,room_type_reserved,lead_time,arrival_month,market_segment_type,repeated_guest,no_of_previous_cancellations,no_of_previous_bookings_not_canceled,avg_price_per_room,no_of_special_requests,booking_status
0,2,0,1,2,Meal Plan 1,0,Room_Type 1,224,10,Offline,0,0,0,65.00,0,Not_Canceled
1,2,0,2,3,Not Selected,0,Room_Type 1,5,11,Online,0,0,0,106.68,1,Not_Canceled
2,1,0,2,1,Meal Plan 1,0,Room_Type 1,1,2,Online,0,0,0,60.00,0,Canceled
3,2,0,0,2,Meal Plan 1,0,Room_Type 1,211,5,Online,0,0,0,100.00,0,Canceled
4,2,0,1,1,Not Selected,0,Room_Type 1,48,4,Online,0,0,0,94.50,0,Canceled
